In [ ]:
import face_recognition
import cv2
import numpy as np

In [ ]:
video = "files/jimmy_fallon_video.mp4"
video_capture = cv2.VideoCapture(video)
print("video file capture...")
image_person1 = face_recognition.\
        load_image_file("files/Jimmy_Fallon_2013.jpeg")
face_encoding1 = face_recognition.\
        face_encodings(image_person1)[0]

known_face_encodings = [face_encoding1]
known_face_names = ["Jimmy Fallon"]
print("face encoding created.")

In [ ]:
process_frame = True
while True:
    ret, frame = video_capture.read()
    
    if process_frame:
        small_frame = cv2.resize(frame, 
                        (0, 0), fx=0.25, fy=0.25)
        rgb_frame = small_frame[:, :, ::-1]
        
        face_locs = face_recognition.\
            face_locations(rgb_frame)
        face_encodings = face_recognition.\
            face_encodings(rgb_frame, 
                           face_locs)
        
        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.\
                compare_faces(known_face_encodings, 
                              face_encoding)
            name = "?"
            face_distances = face_recognition.\
                face_distance(known_face_encodings, 
                              face_encoding)
            best_match = np.argmin(face_distances)
            if matches[best_match]:
                name = known_face_names[best_match]
                
            face_names.append(name)
            
    process_frame = not process_frame
    
    loc_names = zip(face_locs, face_names)
    for (top,right,bottom,left), name in loc_names:
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        cv2.rectangle(frame, (left, top), 
                          (right, bottom), 
                          (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), 
                      (right, bottom), (0, 0, 255), 
                      cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, 
                    (left + 6, bottom - 6), 
                    font, 1.0, (255, 255, 255), 1)
        
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()